<a href="https://colab.research.google.com/github/F1R3xS1NN3R/75XV2/blob/main/EAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EasyAimLock**
### ***`FIRST DO NOT RUN THIS FROM THE EAL LINK`***
Click File on the top left and "Save a copy in Drive" then run the copy.

Note: This is not on your computer and you are not installing anything. It is on a free cloud computer provided by google. When you close this window google deletes the computer. It is using their GPUs to train the model which are faster than most desktop GPUs and allows you to train models from anywhere with internet access and a browser. Before you start training bookmark the page so you can return to the notebook easily. I suggest keeping the page open while it trains. We have added code to help prevent the colab from disconnecting.

Before you start, prepare your image dataset. If you have already prepared your dataset you can continue on to the **MAIN** section. If you still need to prepare your image dataset expand the hidden cell in this section for instructions. 

---





## **`How to prepare your dataset for training:`**

# 1) Collect Images

> Open EAL and navigate to Preferences/Training and enable CREATE CUSTOM DATASET to automatically collect images in the proper format.

> Only use .png image files in your dataset.

> We recommend more that 1000 images to get started, several thousand would preferable. The more the better.


# 2) Label Your Images

> Unzip the images folder from the EasyAimLock cache folder

> Open a command promt and type LableImg. If the programe does not open, type pip install LabelImg and when it finishes type LabelImg again.

> Inside LabelImg change your format to Yolo!!!!! Select "Open Dir" and navigate to the images folder, then select "Change Save Dir" and set the save directory as the same folder. Then in the View drop down select single class mode and auto save. On the right select Use default label and write the word person.

> To start labeling click the W key on the keyboard and draw a box around each person. Hit W each time you need to create a new box. Make the box close as possible to the people. When you are done hit D on the keyboard to go to the next image. If you need to go back hit A.

Ensure every single images has been labeled without missing a single object

When you are done go through every image again and verify that every single image is labeled.

If you have an image that is confusing or does not contain a person then delete the image.

If the label box for a person is smaller than 32 pixles do not use that image, delete it.

If you are sure evey image is labeled and accurate recompress all of the images and txt files into an eal.zip file

Create a folder on your google share drive root called EAL

Copy the eal.zip file to your google drive EAL/eal.zip

Then continue to MAIN

# MAIN
Set or change the max_batches, if you dont know what that is leave it to default 3000. Then, start by clicking the play arrow next to the cell title and wait for it complete.

`When you start`

You will be asked to verify your google account. Click the link, chose the google account that has the eal.zip file. Copy the private key and close the page. Then come back to this page and use CTRL+V to enter the key into the text field below the link. Hit enter to verify.

Training the model could take several hours. Every 1000 iterations will make a backup of the model in your google drive EAL/backup directory. There will also be an EAL.cfg file in that directory. If you let the training complete all iterations use the best model, if you do not see a best model use the final model. 


In [ ]:
#@title Easy Aim Lock - Trainer
import IPython
from google.colab import output
import re

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Colab Disconnect Prevention: Active")
from google.colab import drive
drive.mount('/content/drive/')

!/usr/local/cuda/bin/nvcc --version
!nvidia-smi

import os
os.environ['GPU_TYPE'] = str(os.popen('nvidia-smi --query-gpu=name --format=csv,noheader').read())

def getGPUArch(argument):
  try:
    argument = argument.strip()
    # All Colab GPUs
    archTypes = {
        "Tesla V100-SXM2-16GB": "-gencode arch=compute_70,code=[sm_70,compute_70]",
        "Tesla K80": "-gencode arch=compute_37,code=sm_37",
        "Tesla T4": "-gencode arch=compute_75,code=[sm_75,compute_75]",
        "Tesla P40": "-gencode arch=compute_61,code=sm_61",
        "Tesla P4": "-gencode arch=compute_61,code=sm_61",
        "Tesla P100-PCIE-16GB": "-gencode arch=compute_60,code=sm_60"

      }
    return archTypes[argument]
  except KeyError:
    return "GPU must be added to GPU Commands"
os.environ['ARCH_VALUE'] = getGPUArch(os.environ['GPU_TYPE'])

print("GPU Type: " + os.environ['GPU_TYPE'])
print("ARCH Value: " + os.environ['ARCH_VALUE'])

%cd /content/
%rm -rf darknet

!git clone https://github.com/roboflow-ai/darknet.git

%cd /content/darknet/
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/g' Makefile
!sed -i "s/ARCH= -gencode arch=compute_60,code=sm_60/ARCH= ${ARCH_VALUE}/g" Makefile
!make

%cd /content/darknet
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.weights
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

!ln -s /content/drive/My\ Drive/ /mygdrive
!ls /mygdrive/EAL

!ls /mygdrive/EAL
%mkdir /mygdrive/EAL/backup
!cp /mygdrive/EAL/eal.zip ../
!unzip ../eal.zip -d /content/darknet/
%cd /content/darknet/

continue_flag = False
max_batches =  10000#@param {type:"integer"}


if os.path.exists('./eal.cfg') and os.path.exists('./eal.weights'):
  with open('./eal.cfg', 'r') as out:
    lines = out.readlines()
    for i, line in enumerate(lines):
      if 'max_batches' in line:
        split_string = line.split("=", 1)
        substring = split_string[0]
        new_max_batch = int(split_string[1]) + max_batches
        replace_line = split_string[0] + ' = ' + str(new_max_batch) +'\n'
        lines[i] = replace_line
        steps1 = .8 * new_max_batch
        steps2 = .9 * new_max_batch
        steps_str = 'steps = ' + str(steps1)+','+str(steps2) +'\n'
        print(steps_str)
        lines[i+2] = steps_str
        continue_flag = True

%cp train/classes.txt data/obj.names
%mkdir data/obj

%cp train/*.png data/obj/
%cp valid/*.png data/obj/

%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 1\n')
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = /mygdrive/EAL/backup')

#import os
with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('train') if f.endswith(('png','jpg'))]:
    out.write('data/obj/' + img + '\n')

#import os
with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('valid') if f.endswith(('png','jpg'))]:
    out.write('data/obj/' + img + '\n')

def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/classes.txt')

steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(steps1)+','+str(steps2)
num_filters = (num_classes + 5) * 3
input_size =  1024#@param {type:"integer"}

if os.path.exists('./cfg/custom-yolov4-tiny-detector.cfg'): os.remove('./cfg/custom-yolov4-tiny-detector.cfg')

if continue_flag:
  if os.path.exists('./yolov4-tiny.conv.29'): os.remove('./yolov4-tiny.conv.29')
  %cp eal.weights yolov4-tiny.conv.29
  with open('./cfg/custom-yolov4-tiny-detector.cfg', 'w') as f:
    for l in lines:
      f.write(l)
  print('Extended training to ' + str(replace_line) + ' batches.')

else:
  cfg_file = f'''
[net]

batch=64
subdivisions=16
width={input_size} #1024
height={input_size} #1024

channels=3
momentum=0.949
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches = {max_batches}
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=1
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 1,2,3
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=1
nms_kind=greedynms
beta_nms=0.6
'''
  with open('./cfg/custom-yolov4-tiny-detector.cfg', 'w') as f:
    f.write(cfg_file)
    #print(cfg_file)
print('////////////// EASY AIM LOCK /////////////')
print('//////////// TRAINING STARTED ////////////')
%cat cfg/custom-yolov4-tiny-detector.cfg
%cp cfg/custom-yolov4-tiny-detector.cfg /mygdrive/EAL/backup/eal.cfg
!./darknet detector train data/obj.data cfg/custom-yolov4-tiny-detector.cfg yolov4-tiny.conv.29 -dont_show -map
if os.path.exists('./chart.png'):
  %cp chart.png /mygdrive/EAL/backup/EAL.png

<IPython.core.display.Javascript object>

Streaming output truncated to the last 5000 lines.
 try to allocate additional workspace_size = 26.22 MB 
 CUDA allocate done! 
Loaded: 2.190573 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 9760 iterations) 
 Last accuracy mAP@0.5 = 99.39 %, best = 99.39 % 
 9101: 0.091198, 0.059549 avg loss, 0.000026 rate, 2.288411 seconds, 582464 images, 1.186392 hours left
Loaded: 0.892776 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 9760 iterations) 
 Last accuracy mAP@0.5 = 99.39 %, best = 99.39 % 
 9102: 0.060289, 0.059623 avg loss, 0.000026 rate, 2.204980 seconds, 582528 images, 1.185713 hours left
Loaded: 0.896898 seconds - performance bottleneck on CPU or Disk HDD/SSD

 (next mAP calculation at 9760 iterations) 
 Last accuracy mAP@0.5 = 99.39 %, best = 99.39 % 
 9103: 0.100471, 0.063708 avg loss, 0.000026 rate, 2.197297 seconds, 582592 images, 1.181584 hours left
Loaded: 0.792930 seconds - performance bottleneck on CPU